In [1]:
!pip install pyspark
!pip install pyngrok

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyngrok import ngrok
spark = SparkSession.builder.appName("DijkstraShortestPath").getOrCreate()
file1 = spark.read.csv("question2_1.txt", sep=",", inferSchema=True, header=False).toDF("source", "destination", "weight")
file2 = spark.read.csv("question2_2.txt", sep=",", inferSchema=True, header=False).toDF("source", "destination", "weight")
edges = file1.union(file2)

edges_rdd = edges.rdd.map(lambda row: (row["source"], (row["destination"], row["weight"])))

start_node = 0
nodes = edges.select("source").distinct().union(edges.select("destination").distinct()).distinct()
distances = {row[0]: float('inf') for row in nodes.collect()}
distances[start_node] = 0
distances_broadcast = spark.sparkContext.broadcast(distances)
for i in range(len(distances)):
    new_distances = (
        edges_rdd.flatMap(lambda edge: [(edge[0], distances_broadcast.value[edge[0]]),
                                        (edge[1][0], distances_broadcast.value[edge[0]] + edge[1][1])])
        .reduceByKey(min)
    )
    new_distances_dict = dict(new_distances.collect())
    for node, dist in new_distances_dict.items():
        distances[node] = dist if dist < distances[node] else distances[node]
    distances_broadcast = spark.sparkContext.broadcast(distances)
distances = {node: float(dist) for node, dist in distances.items()}
distances_df = spark.createDataFrame([Row(node=node, shortest_distance=dist) for node, dist in distances.items()])
distances_list = distances_df.collect()
with open("output_2.txt", "w") as f:
    f.write("node,shortest_distance\n")
    for row in distances_list:
        f.write(f"{row['node']},{row['shortest_distance']}\n")
max_distance_node = distances_df.orderBy("shortest_distance", ascending=False).first()
filtered_distances_df = distances_df.filter(distances_df.node != start_node).orderBy("shortest_distance")
min_distance_node = filtered_distances_df.first()

print(f"Farthest Node: {max_distance_node}")
print(f"Closes Node: {min_distance_node}")



ngrok.set_auth_token("2oY32y1o2wSGEJe5t6a6WZAnWxW_7cMg48xpKAosLShJ5LDdX")
public_url = ngrok.connect(4040)
print(public_url)


Farthest Node: Row(node=3.0, shortest_distance=6.0)
Closes Node: Row(node=22.0, shortest_distance=1.0)
NgrokTunnel: "https://2136-34-44-50-154.ngrok-free.app" -> "http://localhost:4040"
